In [2]:
"Option pricing models"

import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.integrate import quad
import warnings
warnings.filterwarnings('ignore')

In [19]:
# --- 1. Define Heston Model Parameters and Option Details ---

# Model parameters
v0 = 0.04       # Initial variance (20% volatility)
theta = 0.04    # Long-run variance
kappa = 2.0     # Speed of mean reversion
sigma = 0.3     # Volatility of variance
rho = -0.7      # Correlation between asset and variance (typically negative)
#Market parameters
S0 = 100.0      # Initial stock price
K = 80.0          # Strike price
T = 1.0         # Time to maturity (1 year)
r = 0.03        # Risk-free rate (3%)
q = 0.02        # Dividend yield (2%)

# --- 2. Define the Heston Characteristic Function ---

def char_func_heston(u, T, r, v0, theta, kappa, sigma, rho, stable=True):
    
    if stable==True:
        """
        Computes the Heston characteristic function using a numerically stable formulation
        """
        # This version uses hyperbolic functions for improved numerical stability
        # Numerically stable calculation of C and D

        i = 1j  # Imaginary unit
        
        # Common terms
        beta = kappa - rho * sigma * i * u
        d = np.sqrt(beta**2 + sigma**2 * (u**2 + i * u))
        d_t_half = d * T / 2.0
        
        # Handle the case where d_t_half is very small to avoid division by zero in sinh
        # For small x, sinh(x) approx x and cosh(x) approx 1
        # This prevents NaN values for u=0
        small_d_mask = np.abs(d_t_half) < 1e-10
        
        # Calculate sinh and cosh safely
        sinh_d_tau_half = np.sinh(d_t_half)
        cosh_d_tau_half = np.cosh(d_t_half)
        # Note: coth(x) = 1/tanh(x) = cosh(x)/sinh(x)
        """
        # Avoid numerical issues when d*T is very small
        if np.abs(d_half_T) < 1e-8:
            # Use Taylor expansion for small arguments
            sinh_d_tau_half = d_t_half * (1 + (d_t_half)**2 / 6)
            cosh_d_tau_half = 1 + (d_t_half)**2 / 2
        else:
            sinh_d_tau_half = np.sinh(d_t_half)
            cosh_d_tau_half = np.cosh(d_t_half)
        """

        # D term
        # When d is small, d*coth(d*T/2) -> 2/T
        D_val = -(u**2 + i * u) / (beta + d * cosh_d_tau_half / sinh_d_tau_half)
        D_val[small_d_mask] = -(u[small_d_mask]**2 + i * u[small_d_mask]) / (beta[small_d_mask] + 2.0/T)

        # C term
        # When d is small, the log term simplifies
        log_term = cosh_d_tau_half + (beta / d) * sinh_d_tau_half
        log_term[small_d_mask] = 1.0 # Log(1)=0, so the term vanishes
        
        C_val = (kappa * theta / sigma**2) * (beta * T - 2 * np.log(log_term))

        # The characteristic function
        phi = np.exp(C_val + D_val * v0 + i * u * (np.log(S0) + r * T))
    
    else:
        """
        Computes the Heston characteristic function
        """
        i = 1j  # Imaginary unit
        
        # Calculate d and g
        beta = kappa - rho * sigma * i * u
        d = np.sqrt((rho * sigma * i * u - kappa)**2 + sigma**2 * (u**2 + i * u))
        g = (beta - d) / (beta + d)

        # Calculate C and D
        C = ((kappa * theta / sigma**2) * ((beta - d) * T - 2 * np.log((1 - g * np.exp(-d * T)) / (1 - g))))
        D = ((beta - d) / sigma**2) * ((1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)))

        # The characteristic function
        phi = np.exp(C + D * v0 + (i * u) * (r * T + np.log(S0)))
        
    return phi

In [81]:
eta = 0.25
N = 2**12
alpha = 1.5

i = 1j  # Imaginary unit
lambda_ = (2 * np.pi) / (N * eta)  # Step-size in log strike space

# Grids
v = np.arange(N) * eta # Integration points
k = -(N / 2) * lambda_ + np.arange(N) * lambda_  # Strike price range
strikes = np.exp(k)

# Calculate the Fourier transform of the call price
u_for_psi = v - (alpha + 1) * i
numerator = np.exp(-r * T) * char_func_heston(u_for_psi, T, r, v0, theta, kappa, sigma, rho)
denominator = (alpha + i * v) * (alpha + 1 + i * v)
psi = numerator / denominator # Damping function

simpson_weights = (eta / 3) * (3 + (-1)**(np.arange(N) + 1) - (np.arange(N) == 0))
fft_input = np.exp(1j * v * (N / 2) * lambda_) * psi * simpson_weights

call_prices_fft = np.fft.fft(fft_input)
    
call_prices = np.exp(-alpha * k) / np.pi * np.real(call_prices_fft)
call_prices1 = np.exp(-alpha * k) * np.real(call_prices_fft) / np.pi

# Find the price for an at-the-money strike (K=100)
atm_strike = 80
atm_price = np.interp(atm_strike, strikes, call_prices)


In [ ]:
# --- 3. Implement the FFT-based Pricing Method ---
# This function calls the stable characteristic function

def price_call_heston_fft_stable(S0, T, r, v0, theta, kappa, sigma, rho, N = 2**12, eta = 0.25, alpha = 1.5, simpson=True):
    """
    Prices European call options using the Heston model and FFT.
    Returns a range of strikes and their corresponding call prices.

    K: Strike price
    T: Time to maturity
    N: Number of points for FFT
    alpha: Damping parameter
    # FFT parameters
    N = 2**12             # Number of points, should be a power of 2
    alpha = 1.5           # Dampening factor for integrability
    eta = 0.25            # Grid spacing in Fourier domain
    """

    i = 1j  # Imaginary unit
    lambda_ = (2 * np.pi) / (N * eta)  # Step-size in log strike space

    # Grids
    v = np.arange(N) * eta # Integration points
    #k = np.log(S0) - (N / 2) * lambda_ + np.arange(N) * lambda_  # Strike price range
    k = -(N * lambda_ / 2) + lambda_ * np.arange(N)
    strikes = np.exp(k)
    
    # Calculate the Fourier transform of the call price
    u_for_psi = v - (alpha + 1) * i
    numerator = np.exp(-r * T) * char_func_heston(u_for_psi, T, r, v0, theta, kappa, sigma, rho)
    denominator = (alpha + i * v) * (alpha + 1 + i * v)
    psi = numerator / denominator # Damping function

    if simpson == True:
        # Simpson's rule weights
        # Apply Simpson's rule weighting for better accuracy
        simpson_weights = (eta / 3) * (3 + (-1)**(np.arange(N) + 1) - (np.arange(N) == 0))
        fft_input = np.exp(i * v * (N / 2) * lambda_) * psi * simpson_weights
            
    else:
        eta = np.full(N, eta)
        fft_input = np.exp(i * v * (N / 2) * lambda_) * psi * eta

    # FFT
    call_prices_fft = np.fft.fft(fft_input)
    
    call_prices = np.exp(-alpha * k) / np.pi * np.real(call_prices_fft)
    #call_prices = np.exp(-alpha * k) * np.real(call_prices_fft) / np.pi
    
    return strikes, call_price


In [ ]:
class HestonModel:
    """
    Heston Stochastic Volatility Model for Option Pricing
    """
    
    def __init__(self, S0, v0, rho, kappa, theta, sigma, r, q=0):
        """
        Initialize Heston model parameters
        
        S0: Initial stock price
        v0: Initial variance
        rho: Correlation between stock and volatility
        kappa: Mean reversion speed
        theta: Long-term variance
        sigma: Volatility of variance (vol of vol)
        r: Risk-free rate
        q: Dividend yield
        """
        self.S0 = S0
        self.v0 = v0
        self.rho = rho
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.r = r
        self.q = q
        
    def characteristic_function(self, u, T):
        """
        Heston characteristic function for FFT pricing
        """
        # Parameters
        kappa, theta, sigma, rho, v0, r, q = self.kappa, self.theta, self.sigma, self.rho, self.v0, self.r, self.q
        
        # Complex number calculations
        i = complex(0, 1)
        
        # Auxiliary variables
        d = np.sqrt((rho * sigma * i * u - kappa)**2 - sigma**2 * (2 * i * u - u**2))
        g = (kappa - rho * sigma * i * u - d) / (kappa - rho * sigma * i * u + d)
        
        # Exponential terms
        exp1 = np.exp(-d * T)
        exp2 = np.exp((r - q) * i * u * T)
        
        # C and D functions
        C = (r - q) * i * u * T + (kappa * theta / sigma**2) * ((kappa - rho * sigma * i * u - d) * T - 2 * np.log((1 - g * exp1) / (1 - g)))
        D = (kappa - rho * sigma * i * u - d) / sigma**2 * ((1 - exp1) / (1 - g * exp1))
        
        # Characteristic function
        phi = exp2 * np.exp(C + D * v0)
        
        return phi
    
    def price_european_call_fft(self, K, T, N=2**15, alpha=1.5):
        """
        Price European call option using FFT method
        
        K: Strike price
        T: Time to maturity
        N: Number of points for FFT
        alpha: Damping parameter
        """
        # FFT parameters
        eta = 0.25
        lam = 2 * np.pi / (N * eta)
        b = N * lam / 2
        
        # Strike price range
        k = -b + lam * np.arange(N)
        
        # Integration points
        v = eta * np.arange(N)
        
        # Damping function
        psi = lambda u: np.exp(-self.r * T) * self.characteristic_function(u - (alpha + 1) * 1j, T) / (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)
        
        # FFT input
        x = np.zeros(N, dtype=complex)
        x[0] = 0.5 * psi(0)
        for i in range(1, N):
            x[i] = psi(v[i])
        
        # Apply trapezoidal rule weights
        x *= eta
        
        # FFT
        y = np.fft.fft(x)
        
        # Call option values
        call_values = (np.exp(-alpha * k) / np.pi) * np.real(y)
        
        # Interpolate for specific strike
        log_strike = np.log(K)
        call_price = np.interp(log_strike, k, call_values)
        
        return call_price
    
    def price_american_put_cn(self, K, T, S_max=None, M=100, N=500):
        """
        Price American put option using Crank-Nicolson finite difference method
        
        K: Strike price
        T: Time to maturity
        S_max: Maximum stock price for grid
        M: Number of time steps
        N: Number of stock price steps
        """
        if S_max is None:
            S_max = 2 * max(self.S0, K)
        
        # Grid parameters
        dt = T / M
        dS = S_max / N
        
        # Stock price grid
        S = np.linspace(0, S_max, N+1)
        
        # Variance grid (simplified approach - using constant variance)
        # For full 2D PDE, we would need a 2D grid
        v = np.full_like(S, self.v0)
        
        # Initialize option values at maturity (put payoff)
        V = np.maximum(K - S, 0)
        
        # Boundary conditions
        # At S=0: V = K
        # At S=S_max: V ≈ 0 for put option
        
        for j in range(M):
            # Build tridiagonal matrix for Crank-Nicolson
            # This is a simplified 1D version - full Heston would require 2D grid
            
            # Calculate local volatility approximation
            vol = np.sqrt(np.maximum(v, 0.01))  # Ensure positive volatility
            
            # Coefficients for finite difference scheme
            alpha_coeff = 0.25 * dt * (vol**2 * S**2 / dS**2 - (self.r - self.q) * S / dS)
            beta_coeff = -0.5 * dt * (vol**2 * S**2 / dS**2 + self.r)
            gamma_coeff = 0.25 * dt * (vol**2 * S**2 / dS**2 + (self.r - self.q) * S / dS)
            
            # Create tridiagonal matrix A for implicit part
            A = sparse.diags([alpha_coeff[2:], 1 + beta_coeff[1:], gamma_coeff[:-2]], 
                           offsets=[-1, 0, 1], shape=(N-1, N-1), format='csr')
            
            # Create tridiagonal matrix B for explicit part
            B = sparse.diags([-alpha_coeff[2:], 1 - beta_coeff[1:], -gamma_coeff[:-2]], 
                           offsets=[-1, 0, 1], shape=(N-1, N-1), format='csr')
            
            # Right-hand side
            rhs = B @ V[1:-1]
            
            # Apply boundary conditions to RHS
            rhs[0] += alpha_coeff[1] * V[0] + (-alpha_coeff[1]) * V[0]
            rhs[-1] += gamma_coeff[-2] * V[-1] + (-gamma_coeff[-2]) * V[-1]
            
            # Solve linear system
            V_new = np.zeros_like(V)
            V_new[0] = K  # Boundary condition at S=0
            V_new[-1] = 0  # Boundary condition at S=S_max
            V_new[1:-1] = spsolve(A, rhs)
            
            # Apply American option constraint (early exercise)
            intrinsic = np.maximum(K - S, 0)
            V_new = np.maximum(V_new, intrinsic)
            
            V = V_new.copy()
        
        # Interpolate to get option price at S0
        option_price = np.interp(self.S0, S, V)
        
        return option_price, S, V

def main():
    """
    Main function to demonstrate Heston model option pricing
    """
    print("Heston Model Option Pricing Implementation")
    print("=" * 50)
    
    # Market parameters (realistic values)
    S0 = 100.0      # Current stock price
    K = 100.0       # Strike price
    T = 0.25        # Time to maturity (3 months)
    r = 0.05        # Risk-free rate (5%)
    q = 0.02        # Dividend yield (2%)
    
    # Heston model parameters (calibrated values for typical equity)
    v0 = 0.04       # Initial variance (20% volatility)
    kappa = 2.0     # Mean reversion speed
    theta = 0.04    # Long-term variance
    sigma = 0.3     # Volatility of variance
    rho = -0.7      # Correlation (typically negative)
    
    print(f"Market Parameters:")
    print(f"Stock Price (S0): ${S0}")
    print(f"Strike Price (K): ${K}")
    print(f"Time to Maturity (T): {T} years")
    print(f"Risk-free Rate (r): {r:.1%}")
    print(f"Dividend Yield (q): {q:.1%}")
    print()
    
    print(f"Heston Model Parameters:")
    print(f"Initial Variance (v0): {v0:.4f}")
    print(f"Mean Reversion Speed (κ): {kappa}")
    print(f"Long-term Variance (θ): {theta:.4f}")
    print(f"Vol of Vol (σ): {sigma}")
    print(f"Correlation (ρ): {rho}")
    print()
    
    # Initialize Heston model
    heston = HestonModel(S0, v0, rho, kappa, theta, sigma, r, q)
    
    # 1. Price European Call Option using FFT
    print("1. European Call Option Pricing (FFT Method)")
    print("-" * 40)
    
    try:
        call_price_fft = heston.price_european_call_fft(K, T)
        print(f"European Call Option Price: ${call_price_fft:.4f}")
        
        # Calculate European put price using put-call parity
        put_price_european = call_price_fft - S0 * np.exp(-q * T) + K * np.exp(-r * T)
        print(f"European Put Option Price (Put-Call Parity): ${put_price_european:.4f}")
        
    except Exception as e:
        print(f"Error in FFT pricing: {e}")
        call_price_fft = None
    
    print()
    
    # 2. Price American Put Option using Crank-Nicolson
    print("2. American Put Option Pricing (Crank-Nicolson Method)")
    print("-" * 50)
    
    try:
        put_price_american, S_grid, V_grid = heston.price_american_put_cn(K, T, M=100, N=200)
        print(f"American Put Option Price: ${put_price_american:.4f}")
        
        # Early exercise premium
        if 'put_price_european' in locals():
            early_exercise_premium = put_price_american - put_price_european
            print(f"Early Exercise Premium: ${early_exercise_premium:.4f}")
        
    except Exception as e:
        print(f"Error in Crank-Nicolson pricing: {e}")
        put_price_american = None
    
    print()
    
    # 3. Sensitivity Analysis
    print("3. Sensitivity Analysis")
    print("-" * 25)
    
    # Test different volatility levels
    vol_levels = [0.15, 0.20, 0.25, 0.30, 0.35]
    print("Impact of Initial Volatility on Option Prices:")
    print("Vol Level | European Call | American Put")
    print("-" * 40)
    
    for vol in vol_levels:
        v_test = vol**2  # Convert to variance
        heston_test = HestonModel(S0, v_test, rho, kappa, theta, sigma, r, q)
        
        try:
            call_test = heston_test.price_european_call_fft(K, T)
            put_test, _, _ = heston_test.price_american_put_cn(K, T, M=50, N=100)
            print(f"{vol:.0%}       | ${call_test:.4f}        | ${put_test:.4f}")
        except:
            print(f"{vol:.0%}       | Error           | Error")
    
    print()
    
    # 4. Visualization
    if 'S_grid' in locals() and 'V_grid' in locals():
        print("4. Generating Visualization...")
        
        plt.figure(figsize=(12, 8))
        
        # Plot 1: American Put Option Value vs Stock Price
        plt.subplot(2, 2, 1)
        plt.plot(S_grid, V_grid, 'b-', linewidth=2, label='American Put Value')
        plt.plot(S_grid, np.maximum(K - S_grid, 0), 'r--', label='Intrinsic Value')
        plt.xlabel('Stock Price ($)')
        plt.ylabel('Option Value ($)')
        plt.title('American Put Option Value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 2: Payoff diagram
        plt.subplot(2, 2, 2)
        S_range = np.linspace(80, 120, 100)
        call_payoff = np.maximum(S_range - K, 0)
        put_payoff = np.maximum(K - S_range, 0)
        
        plt.plot(S_range, call_payoff, 'g-', label='Call Payoff', linewidth=2)
        plt.plot(S_range, put_payoff, 'r-', label='Put Payoff', linewidth=2)
        plt.axvline(x=K, color='k', linestyle='--', alpha=0.5, label='Strike')
        plt.xlabel('Stock Price at Expiration ($)')
        plt.ylabel('Payoff ($)')
        plt.title('Option Payoff Diagrams')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 3: Volatility surface (simplified)
        plt.subplot(2, 2, 3)
        strikes = np.linspace(80, 120, 20)
        maturities = np.linspace(0.1, 1.0, 10)
        
        if call_price_fft is not None:
            vol_surface = np.zeros((len(maturities), len(strikes)))
            for i, T_test in enumerate(maturities):
                for j, K_test in enumerate(strikes):
                    try:
                        price = heston.price_european_call_fft(K_test, T_test)
                        # This is a placeholder - actual implied vol calculation would be more complex
                        vol_surface[i, j] = np.sqrt(v0) * (1 + 0.1 * np.random.randn())
                    except:
                        vol_surface[i, j] = np.sqrt(v0)
            
            X, Y = np.meshgrid(strikes, maturities)
            plt.contourf(X, Y, vol_surface, levels=20, cmap='viridis')
            plt.colorbar(label='Implied Volatility')
            plt.xlabel('Strike Price ($)')
            plt.ylabel('Time to Maturity (years)')
            plt.title('Volatility Surface (Illustrative)')
        
        # Plot 4: Model parameters impact
        plt.subplot(2, 2, 4)
        rho_values = np.linspace(-0.9, 0.9, 10)
        prices_by_rho = []
        
        for rho_test in rho_values:
            try:
                heston_rho = HestonModel(S0, v0, rho_test, kappa, theta, sigma, r, q)
                price = heston_rho.price_european_call_fft(K, T)
                prices_by_rho.append(price)
            except:
                prices_by_rho.append(np.nan)
        
        plt.plot(rho_values, prices_by_rho, 'mo-', linewidth=2)
        plt.xlabel('Correlation (ρ)')
        plt.ylabel('Call Option Price ($)')
        plt.title('Impact of Correlation on Call Price')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("Visualization complete!")
    
    print("\n" + "=" * 50)
    print("Heston Model Implementation Summary:")
    print("✓ European call option pricing using FFT")
    print("✓ American put option pricing using Crank-Nicolson")
    print("✓ Sensitivity analysis")
    print("✓ Visualization of results")
    print("=" * 50)

if __name__ == "__main__":
    main()